In [ ]:
import kaolin
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps
import interactive_polyscope
from scipy.spatial import Voronoi, voronoi_plot_2d
from io import BytesIO
from PIL import Image
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.sdf_MLP as mlp
import sdfpred_utils.sdf_functions as sdf
import sdfpred_utils.loss_functions as lf
import trimesh
from scipy.spatial import Delaunay, Voronoi


#cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))

#default tensor types
torch.set_default_tensor_type(torch.cuda.DoubleTensor)

multires = 2
input_dims = 3
lr_sites = 0.03
lr_model = 0.0003
iterations = 5000
save_every = 100
max_iter = 100
#learning_rate = 0.03
destination = "./images/autograd/3D/"


Using device:  NVIDIA GeForce RTX 3090


In [ ]:
#currently sites are between -5 and 5 in all 3 dimensions
# check if sites exists
#num_centroids = 16*16*16
num_centroids = 8*8*8
site_fp = f'sites_{num_centroids}.pt'

if os.path.exists(site_fp):
    sites = torch.load(site_fp)
else:
    sites = su.createCVTgrid(num_centroids=num_centroids, dimensionality=input_dims)
    #save the initial sites torch tensor
    torch.save(sites, site_fp)


def plot_voronoi_3d(sites, xlim=5, ylim=5, zlim=5):
    import numpy as np
    import pyvoro
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    # initialize random number generator
    rng = np.random.default_rng(11)
    # create a set of points in 3D
    points = sites.detach().cpu().numpy()

    # use pyvoro to compute the Voronoi tessellation
    # the second argument gives the the axis limits in x,y and z direction
    # in this case all between 0 and 1.
    # the third argument gives "dispersion = max distance between two points
    # that might be adjacent" (not sure how exactly this works)
    voronoi = pyvoro.compute_voronoi(points,[[-xlim,xlim],[-ylim,ylim],[-zlim,zlim]],1)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # for each Voronoi cell, plot all the faces of the corresponding polygon
    for vnoicell in voronoi:
        faces = []
        # the vertices are the corner points of the Voronoi cell
        vertices = np.array(vnoicell['vertices'])
        # cycle through all faces of the polygon
        for face in vnoicell['faces']:
            faces.append(vertices[np.array(face['vertices'])])
            
        # join the faces into a 3D polygon
        polygon = Poly3DCollection(faces, alpha=0.5, 
                                facecolors=rng.uniform(0,1,3),
                                linewidths=0.5,edgecolors='black')
        ax.add_collection3d(polygon)
    
    ax.set_xlim([-xlim,xlim])
    ax.set_ylim([-ylim,ylim])
    ax.set_zlim([-zlim,zlim])
            
    plt.show()

#plot_voronoi_3d(sites)

In [ ]:
ps.init()
#ps_cloud = ps.register_point_cloud("initial_cvt_grid",sites.detach().cpu().numpy())

In [ ]:

# Load the mesh
mesh = ["bunny", "Resources/stanford-bunny.obj"]
bunny = trimesh.load(mesh[1])

# Step 1: Get current bounding box
min_bound = bunny.bounds[0]  # Min (x, y, z)
max_bound = bunny.bounds[1]  # Max (x, y, z)

# Step 2: Compute scale factor
current_size = max_bound - min_bound  # Size in each dimension
target_size = 8  # Because we want [-5, 5], the total size is 10

scale_factor = target_size / np.max(current_size)  # Scale based on the largest dimension

# Step 3: Compute new center after scaling
new_vertices = bunny.vertices * scale_factor  # Scale the vertices
new_min = np.min(new_vertices, axis=0)
new_max = np.max(new_vertices, axis=0)
new_center = (new_min + new_max) / 2  # New center after scaling

# Step 4: Compute translation to center the bunny at (0,0,0)
translation = -new_center  # Move to the origin

# Step 5: Apply transformation (scaling + translation)
bunny.vertices = new_vertices + translation

target_points = bunny.sample(16*16*16)
target_points = torch.tensor(target_points, device=device)

ps_cloud = ps.register_point_cloud("Target_points",target_points.detach().cpu().numpy())


In [ ]:
import matplotlib
import os

model = mlp.Decoder(multires=multires, input_dims=input_dims).to(device)
model_path = 'models_resources/pretrained_sphere_small.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print('loaded model')
else:
    print("no model found, pretraining")
    model.pre_train_sphere(3000)
    torch.save(model.state_dict(),model_path)
    
def polyscope_sdf(model):
    # Render the SDF as an implicit surface (zero-level set)
    def model_sdf(pts):
        pts_tensor = torch.tensor(pts, dtype=torch.float64, device=device)
        sdf_values = model(pts_tensor)
        sdf_values_np = sdf_values.detach().cpu().numpy().flatten()  # Convert to NumPy
        
        return sdf_values_np

    ps.render_implicit_surface("SDF Surface", model_sdf, mode="sphere_march", enabled=True)



loaded model


In [ ]:
#Todo this should also work in 2d, might want to adjust the simplex iteration
def get_delaunay_neighbors_list_3d_and_all_voronoi_vertices_index(sites):
    # Detach and convert to NumPy for Delaunay triangulation
    points_np = sites.detach().cpu().numpy()
    
    # Compute the Delaunay tessellation
    tri = Delaunay(points_np)
    vor = Voronoi(points_np)
    
    # # Find the neighbors of each point
    neighbors = {i: set() for i in range(len(points_np))}
    for simplex in tri.simplices:  # Each simplex is a tetrahedron (4 points)
        for i in range(4):  # Iterate over each of the 4 vertices
            for j in range(i + 1, 4):  # Connect each pair of points
                neighbors[simplex[i]].add(simplex[j])
                neighbors[simplex[j]].add(simplex[i])

    # Convert sets to lists for easier use
    neighbors = {key: list(value) for key, value in neighbors.items()}
    
    n2 = vor.ridge_points    
    voronoi_vertices_sites = tri.simplices
    
    #todo remove neighbors
    
    return neighbors, n2, voronoi_vertices_sites

#todo deprecate because im dumb
def compute_vertices_index_3d(neighbors):
    vertices_index_to_compute = []
    for site, adjacents in neighbors.items():
        for i in adjacents:
            for n in adjacents:
                if n != site and n != i and n in neighbors[i]:  # Ensuring a triangle exists
                    for m in adjacents:
                        if m != site and m != i and m != n and m in neighbors[i] and m in neighbors[n]:
                            # We now have 4 mutually connected points forming a tetrahedron
                            vertices_index_to_compute.append([i, site, n, m])
    
    # Deduplicate tetrahedra (avoid different orderings of the same set)
    seen_tetrahedra = set()
    filtered_tetrahedra = []

    for tetrahedron in vertices_index_to_compute:
        canonical_tetrahedron = tuple(sorted(tetrahedron, key=str))  # Sort for uniqueness
        if canonical_tetrahedron not in seen_tetrahedra:
            seen_tetrahedra.add(canonical_tetrahedron)
            filtered_tetrahedra.append(tetrahedron)
    
    return filtered_tetrahedra


def compute_zero_crossing_vertices_3d(sites, model):
    """
    Computes the indices of the sites composing vertices where neighboring sites have opposite or zero SDF values.

    Args:
        sites (torch.Tensor): (N, D) tensor of site positions.
        model (callable): Function or neural network that computes SDF values.

    Returns:
        zero_crossing_vertices_index (list of triplets): List of sites indices (si, sj, sk) where atleast 2 sites have opposing SDF signs.
    """
    # Compute Delaunay neighbors
    neighbors, n2, all_tetrahedra = get_delaunay_neighbors_list_3d_and_all_voronoi_vertices_index(sites)

    # Compute SDF values for all sites
    sdf_values = model(sites)  # Assuming model outputs (N, 1) or (N,) tensor


    #todo, probably using voronoi.ridgeopints is faster
    # Find pairs of neighbors with opposing SDF values
    zero_crossing_pairs = set()
    for i, adjacents in neighbors.items():
        for j in adjacents:
            if i < j:  # Avoid duplicates
                sdf_i, sdf_j = sdf_values[i].item(), sdf_values[j].item()
                if sdf_i * sdf_j <= 0:  # Opposing signs or one is zero
                    zero_crossing_pairs.add((i, j))
   
    zero_crossing_vertices_index = []
    
    for tetrahedron in all_tetrahedra:
        tetrahedron_pairs = {
            (tetrahedron[0], tetrahedron[1]), (tetrahedron[0], tetrahedron[2]), (tetrahedron[0], tetrahedron[3]),
            (tetrahedron[1], tetrahedron[2]), (tetrahedron[1], tetrahedron[3]), (tetrahedron[2], tetrahedron[3])
        }
        if any(pair in zero_crossing_pairs for pair in tetrahedron_pairs):
            zero_crossing_vertices_index.append(tetrahedron)


    print("zero_crossing_vertices_index_LIST", len(zero_crossing_vertices_index))
    print("zero_crossing_pairs_bisectors_LIST", len(zero_crossing_pairs))
    zero_crossing_pairs = list(zero_crossing_pairs)
    zero_crossing_pairs = torch.tensor(np.array(zero_crossing_pairs), device=device)
    


    #todo fix this as its the good way to do it
    # neighbors = torch.tensor(np.array(n2), device=device)
    # all_tetrahedra = torch.tensor(np.array(all_tetrahedra), device=device)
    
    # # Extract the SDF values for each site in the pair
    # sdf_i = sdf_values[neighbors[:, 0]]  # First site in each pair
    # sdf_j = sdf_values[neighbors[:, 1]]  # Second site in each pair
    # # Find the indices where SDF values have opposing signs or one is zero
    # mask = (sdf_i * sdf_j <= 0).squeeze()
    # zero_crossing_pairs = neighbors[mask]

######################################### Todo HERE #########################################
    # # Generate all unique edges for each tetrahedron
    # tetrahedron_edges = torch.stack([
    #     all_tetrahedra[:, [0, 1]], all_tetrahedra[:, [0, 2]], all_tetrahedra[:, [0, 3]],
    #     all_tetrahedra[:, [1, 2]], all_tetrahedra[:, [1, 3]], all_tetrahedra[:, [2, 3]]
    # ], dim=1)  # Shape: (T, 6, 2)

    # # Sort each edge to ensure consistent ordering
    # tetrahedron_edges = torch.sort(tetrahedron_edges, dim=2)[0]  # Shape: (T, 6, 2)
    # zero_crossing_pairs = torch.sort(zero_crossing_pairs, dim=1)[0]  # Shape: (Z, 2)

    # # Efficient membership check using `torch.isin`
    # tetrahedron_edges_flat = tetrahedron_edges.view(-1, 2)  # Flatten to (6T, 2)
    # matches = torch.isin(tetrahedron_edges_flat, zero_crossing_pairs).all(dim=1)  # (6T,)

    # # Reshape back to (T, 6) and check if any edge in a tetrahedron has a match
    # match_per_tetrahedron = matches.view(-1, 6).any(dim=1)  # (T,)

    # # Select tetrahedra that contain at least one zero-crossing edge
    # zero_crossing_vertices_index = all_tetrahedra[match_per_tetrahedron]

    ################################################################################
    
    print("zero_crossing_vertices_index_TORCH",zero_crossing_vertices_index.shape)


    #print("zero_crossing_pairs_TORCH", zero_crossing_pairs.shape)
    
    
    return zero_crossing_vertices_index, zero_crossing_pairs

# def compute_vertex_3d(s_i, s_j, s_k, s_l):
#     """
#     Computes the circumcenter of a tetrahedron given four sites in 3D.

#     Args:
#         s_i, s_j, s_k, s_l (torch.Tensor): 3D coordinates of four sites (shape: (3,)).

#     Returns:
#         torch.Tensor: The circumcenter (x, y, z).
#     """
#     # Convert to coordinate form
#     x_i, y_i, z_i = s_i
#     x_j, y_j, z_j = s_j
#     x_k, y_k, z_k = s_k
#     x_l, y_l, z_l = s_l

#     # Compute squared norms
#     s_i2 = x_i**2 + y_i**2 + z_i**2
#     s_j2 = x_j**2 + y_j**2 + z_j**2
#     s_k2 = x_k**2 + y_k**2 + z_k**2
#     s_l2 = x_l**2 + y_l**2 + z_l**2

#     # Construct matrix system
#     A = torch.tensor([
#         [x_i, y_i, z_i, 1],
#         [x_j, y_j, z_j, 1],
#         [x_k, y_k, z_k, 1],
#         [x_l, y_l, z_l, 1]
#     ], dtype=torch.float32)

#     Dx = torch.tensor([
#         [s_i2, y_i, z_i, 1],
#         [s_j2, y_j, z_j, 1],
#         [s_k2, y_k, z_k, 1],
#         [s_l2, y_l, z_l, 1]
#     ], dtype=torch.float32)

#     Dy = torch.tensor([
#         [x_i, s_i2, z_i, 1],
#         [x_j, s_j2, z_j, 1],
#         [x_k, s_k2, z_k, 1],
#         [x_l, s_l2, z_l, 1]
#     ], dtype=torch.float32)

#     Dz = torch.tensor([
#         [x_i, y_i, s_i2, 1],
#         [x_j, y_j, s_j2, 1],
#         [x_k, y_k, s_k2, 1],
#         [x_l, y_l, s_l2, 1]
#     ], dtype=torch.float32)

#     # Compute determinants
#     detA = torch.det(A)
#     detDx = torch.det(Dx)
#     detDy = torch.det(Dy)  # todo, removed Negative due to orientation
#     detDz = torch.det(Dz)

#     # Compute circumcenter coordinates
#     x = 0.5 * (detDx / detA)
#     y = 0.5 * (detDy / detA)
#     z = 0.5 * (detDz / detA)

#     #todo -y is weird but always gives the right answer when comparing with vor.vertices
#     #maybe error in matrix computation
#     return torch.stack([x, y, z])

# def compute_all_vertices_3d(sites, vertices_to_compute):
#     """
#     Computes all Voronoi vertices for a given set of tetrahedra.

#     Args:
#         sites (torch.Tensor): (N, 3) tensor of site positions.
#         vertices_to_compute (list of quadruplets): List of indices forming tetrahedra.

#     Returns:
#         torch.Tensor: Computed Voronoi vertices.
#     """
#     vertices = []
    
#     for quadruplet in vertices_to_compute:
#         si = sites[quadruplet[0]]
#         sj = sites[quadruplet[1]]
#         sk = sites[quadruplet[2]]
#         sl = sites[quadruplet[3]]

#         # Compute circumcenter for the tetrahedron
#         v = compute_vertex_3d(si, sj, sk, sl)
#         vertices.append(v)

#     # Stack all vertices into a single tensor
#     vertices = torch.stack(vertices)
#     return vertices

def compute_vertices_3d_vectorized(sites, vertices_to_compute):
    """
    Computes the circumcenters of multiple tetrahedra in a vectorized manner.

    Args:
        sites (torch.Tensor): (N, 3) tensor of site positions.
        vertices_to_compute (torch.Tensor): (M, 4) tensor of indices forming tetrahedra.

    Returns:
        torch.Tensor: (M, 3) tensor of computed Voronoi vertices.
    """
    # Extract tetrahedra site coordinates in a batched manner
    tetrahedra = sites[vertices_to_compute]  # Shape: (M, 4, 3)

    # Compute squared norms of each point
    squared_norms = (tetrahedra ** 2).sum(dim=2, keepdim=True)  # Shape: (M, 4, 1)

    # Construct the 4x4 matrices in batch
    ones_col = torch.ones_like(squared_norms)  # Column of ones for homogeneous coordinates

    A = torch.cat([tetrahedra, ones_col], dim=2)  # Shape: (M, 4, 4)
    Dx = torch.cat([squared_norms, tetrahedra[:, :, 1:], ones_col], dim=2)
    Dy = torch.cat([tetrahedra[:, :, :1], squared_norms, tetrahedra[:, :, 2:], ones_col], dim=2)
    Dz = torch.cat([tetrahedra[:, :, :2], squared_norms, ones_col], dim=2)

    # Compute determinants in batch
    detA = torch.linalg.det(A)  # Shape: (M,)
    detDx = torch.linalg.det(Dx)
    detDy = torch.linalg.det(Dy)  # todo, removed Negative due to orientation
    detDz = torch.linalg.det(Dz)

    # Compute circumcenters
    circumcenters = 0.5 * torch.stack([detDx / detA, detDy / detA, detDz / detA], dim=1)

    return circumcenters  # Shape: (M, 3)




# #Todo its the same in 3d if we only want a point. Do we want the plane ?
# def compute_all_bisectors_3d(sites, bisectors_to_compute):
#     # Initialize an empty tensor for storing bisectors
#     bisectors = []
    
#     for pairs in bisectors_to_compute:
#         si = sites[pairs[0]]
#         sj = sites[pairs[1]]
#         b = (si + sj) / 2
#         bisectors.append(b)

#     # Stack the list of bisectors into a single tensor for easier gradient tracking
#     bisectors = torch.stack(bisectors)
#     return bisectors


def compute_all_bisectors_3d_vectorized(sites, bisectors_to_compute):
    """
    Computes the bisector points for given pairs of sites in 3D.

    Args:
        sites (torch.Tensor): (N, 3) tensor of site positions.
        bisectors_to_compute (torch.Tensor): (M, 2) tensor of index pairs.

    Returns:
        torch.Tensor: (M, 3) tensor of computed bisector points.
    """
    # Extract all site pairs at once
    si = sites[bisectors_to_compute[:, 0]]  # Shape: (M, 3)
    sj = sites[bisectors_to_compute[:, 1]]  # Shape: (M, 3)

    # Compute bisectors in a single vectorized operation
    bisectors = (si + sj) / 2  # Shape: (M, 3)

    return bisectors


#Todo see above
# def compute_all_bisector_planes_3d(sites, bisectors_to_compute):
#     """
#     Computes bisector planes for given site pairs in 3D.

#     Args:
#         sites (torch.Tensor): (N, 3) tensor of site positions.
#         bisectors_to_compute (list of pairs): List of index pairs (i, j).

#     Returns:
#         torch.Tensor: Plane normals (N, 3) and midpoints (N, 3).
#     """
#     midpoints = []
#     normals = []
    
#     for pair in bisectors_to_compute:
#         si = sites[pair[0]]
#         sj = sites[pair[1]]
        
#         midpoint = (si + sj) / 2  # Midpoint
#         normal = sj - si  # Normal direction
        
#         midpoints.append(midpoint)
#         normals.append(normal)

#     # Stack into tensors
#     midpoints = torch.stack(midpoints)
#     normals = torch.stack(normals)
    
#     return normals, midpoints





In [ ]:

cvt_loss_values = []
min_distance_loss_values = []
edge_smoothing_loss_values = []
chamfer_distance_loss_values = []
eikonal_loss_values = []
domain_restriction_loss_values = []
zero_target_points_loss_values = []
loss_values = []

def autograd(sites, model, max_iter=100, stop_train_threshold=1e-6, upsampling=0, lambda_weights = [0.1,1.0,0.1,0.1,1.0,1.0,0.1]):
    optimizer = torch.optim.Adam([
    {'params': [p for _, p in model.named_parameters()], 'lr': lr_model},
    {'params': [sites], 'lr': lr_sites}
], betas=(0.5, 0.999))

    prev_loss = float("inf")
    best_loss = float("inf")
    upsampled = 0.0
    epoch = 0
    lambda_cvt = lambda_weights[0]
    lambda_sdf = lambda_weights[1]
    lambda_min_distance = lambda_weights[2]
    lambda_laplace = lambda_weights[3]
    lamda_chamfer = lambda_weights[4]
    lamda_eikonal = lambda_weights[5]
    lambda_domain_restriction = lambda_weights[6]
    lambda_target_points = lambda_weights[7]
    
    best_sites = sites.clone()
    best_sites.best_loss = best_loss
    
    while epoch <= max_iter:
        optimizer.zero_grad()
        
        vertices_to_compute, bisectors_to_compute = compute_zero_crossing_vertices_3d(sites, model)
        #vertices_to_compute to array before tensor because extremely slow
        
        #vertices = compute_all_vertices_3d(sites, vertices_to_compute)
        vertices = compute_vertices_3d_vectorized(sites, vertices_to_compute)
        print("vertices length: ",len(vertices))
        
        bisectors = compute_all_bisectors_3d_vectorized(sites, bisectors_to_compute)

        
        #combine vertices and bisectors to one tensor for chamfer
        points = torch.cat((vertices, bisectors), 0)

        print("points length: ",len(points))
  

        # Compute losses       
        cvt_loss = lf.compute_cvt_loss(sites)
        #min_distance_loss = min_distance_regularization_for_op_sites(edges,sites)
        #min_distance_loss = lf.sdf_weighted_min_distance_loss(model, sites)
        #edge_smoothing_loss = compute_edge_smoothing_loss(edges, sites, model)
        chamfer_loss = lf.chamfer_distance(target_points, points)
        eikonal_loss = lf.eikonal(model, input_dimensions=input_dims)
        #domain_restriction_loss = lf.domain_restriction(target_points, model)
        
        sdf_values_target_points = model(target_points)[:,0]
        zero_target_points_loss_L2 = torch.mean(sdf_values_target_points**2)
        zero_target_points_loss_L1 = torch.mean(torch.abs(model(target_points)[:, 0]))
        lambda_1, lambda_2 = 0 , 0.99  # Adjust weights as needed
        zero_target_points_loss = lambda_1 * zero_target_points_loss_L1 + lambda_2 * zero_target_points_loss_L2

               
        # Track raw losses (unweighted)
        #cvt_loss_values.append(cvt_loss.item())
        #min_distance_loss_values.append(min_distance_loss.item())
        #edge_smoothing_loss_values.append(edge_smoothing_loss.item())
        chamfer_distance_loss_values.append(chamfer_loss.item())
        eikonal_loss_values.append(eikonal_loss.item())
        #domain_restriction_loss_values.append(domain_restriction_loss.item())
        zero_target_points_loss_values.append(zero_target_points_loss.item())
  
        loss = (
            lambda_cvt * cvt_loss +
            #lambda_min_distance * min_distance_loss + 
            #lambda_laplace * edge_smoothing_loss +
            lamda_chamfer * chamfer_loss +
            lamda_eikonal * eikonal_loss +
            #lambda_domain_restriction * domain_restriction_loss +
            lambda_target_points * zero_target_points_loss
        )
        loss_values.append(loss.item())
        print(f"Epoch {epoch}: loss = {loss.item()}")
                
        loss.backward()
        optimizer.step()
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_epoch = epoch
            best_sites = sites.clone()
            best_sites.best_loss = best_loss
            if upsampled > 0:
                print(f"UPSAMPLED {upsampled} Best Epoch {best_epoch}: Best loss = {best_loss}")
                #return best_sites
        
        if abs(prev_loss - loss.item()) < stop_train_threshold:
            print(f"Converged at epoch {epoch} with loss {loss.item()}")
            #break
        
        prev_loss = loss.item() 
        
        if epoch>100 and (epoch // 100) == upsampled+1 and loss.item() < 0.5 and upsampled < upsampling:
            print("sites length: ",len(sites))
            
            new_sites = su.upsampling_inside(best_sites, model)
            #new_sites = su.adaptive_density_upsampling(best_sites, model)
            print(new_sites)
            sites = su.add_upsampled_sites(best_sites, new_sites)
            sites = sites.detach().requires_grad_(True)
            print("upsampled sites length: ",len(sites))
            
            best_sites = sites.clone()
            best_sites.best_loss = best_loss
            
            optimizer = torch.optim.Adam([{'params': [p for _, p in model.named_parameters()], 'lr': lr_model},
                                          {'params': [sites], 'lr': lr_sites}])
            upsampled += 1.0
          
        if epoch % (max_iter/10) == 0:
            print(f"Epoch {epoch}: loss = {loss.item()}")
            print(f"Best Epoch {best_epoch}: Best loss = {best_loss}")
            #save model and sites
            site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
            model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
            torch.save(model.state_dict(), model_file_path)
            torch.save(sites, site_file_path)
            
        
        epoch += 1           
        
    return best_sites

In [181]:

lambda_weights = [0.2,0,0.1,0,1.000101101000101,0.1,0,2]

lambda_cvt = lambda_weights[0]
lambda_sdf = lambda_weights[1]
lambda_min_distance = lambda_weights[2]
lambda_laplace = lambda_weights[3]
lamda_chamfer = lambda_weights[4]
lambda_eikonal = lambda_weights[5]
lambda_domain_restriction = lambda_weights[6]
lambda_target_points = lambda_weights[7]

max_iter = 100

site_file_path = f'{destination}{mesh[0]}{max_iter}3d_sites_{num_centroids}_chamfer{lamda_chamfer}.npy'
#check if optimized sites file exists
if os.path.exists(site_file_path):
    #import sites
    print("Importing sites")
    sites = np.load(site_file_path)
    sites = torch.from_numpy(sites).to(device).requires_grad_(True)    
else:
    import cProfile, pstats
    import time
    profiler = cProfile.Profile()
    profiler.enable()
    sites = autograd(sites, model, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights)
    
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.print_stats()
    stats.dump_stats(f'{destination}{mesh[0]}{max_iter}_3d_profile_{num_centroids}_chamfer{lamda_chamfer}.prof')
    
    
    sites_np = sites.detach().cpu().numpy()
    np.save(site_file_path, sites_np)
    

print("Sites length: ", len(sites))
print("min sites: ", torch.min(sites))
print("max sites: ", torch.max(sites))
ps_cloud = ps.register_point_cloud("best_optimized_cvt_grid",sites.detach().cpu().numpy())
    
lim=torch.abs(torch.max(sites)).detach().cpu().numpy()*1.1
plot_voronoi_3d(sites,lim,lim,lim)

zero_crossing_vertices_index_LIST 181
zero_crossing_pairs_bisectors_LIST 137
zero_crossing_vertices_index_TORCH torch.Size([565, 4])
zero_crossing_pairs_TORCH torch.Size([137, 2])
vertices length:  565
points length:  702
Epoch 0: loss = 42.82148415630446
Epoch 0: loss = 42.82148415630446
Best Epoch 0: Best loss = 42.82148415630446
zero_crossing_vertices_index_LIST 282
zero_crossing_pairs_bisectors_LIST 206
zero_crossing_vertices_index_TORCH torch.Size([820, 4])
zero_crossing_pairs_TORCH torch.Size([206, 2])
vertices length:  820
points length:  1026
Epoch 1: loss = 42.215307985152194
zero_crossing_vertices_index_LIST 385
zero_crossing_pairs_bisectors_LIST 283
zero_crossing_vertices_index_TORCH torch.Size([1086, 4])
zero_crossing_pairs_TORCH torch.Size([283, 2])
vertices length:  1086
points length:  1369
Epoch 2: loss = 385.785971327791
zero_crossing_vertices_index_LIST 482
zero_crossing_pairs_bisectors_LIST 390
zero_crossing_vertices_index_TORCH torch.Size([1490, 4])
zero_crossing_pa

KeyboardInterrupt: 

In [ ]:
epoch = 100

model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
sites = torch.load(site_file_path)
sites_np = sites.detach().cpu().numpy()
model.load_state_dict(torch.load(model_file_path))
#
#polyscope_sdf(model)
#
print("model", model_file_path)
print("sites", site_file_path)
#ps_cloud = ps.register_point_cloud(f"{epoch} epoch_cvt_grid",sites_np)



model ./images/autograd/3D/bunny100_100_3d_model_13824_chamfer1.00001101000101.pth
sites ./images/autograd/3D/bunny100_100_3d_sites_13824_chamfer1.00001101000101.pth


In [ ]:
def get_zero_crossing_mesh_3d(sites, model):
    sites_np = sites.detach().cpu().numpy()
    vor = Voronoi(sites_np)  # Compute 3D Voronoi diagram

    sdf_values = model(sites)[:, 0].detach().cpu().numpy()  # Compute SDF values

    valid_faces = []  # List of polygonal faces
    used_vertices = set()  # Set of indices for valid vertices

    for (point1, point2), ridge_vertices in zip(vor.ridge_points, vor.ridge_vertices):
        if -1 in ridge_vertices:
            continue  # Skip infinite ridges

        # Check if SDF changes sign across this ridge
        if np.sign(sdf_values[point1]) != np.sign(sdf_values[point2]):
            valid_faces.append(ridge_vertices)
            used_vertices.update(ridge_vertices)

    # **Filter Voronoi vertices**
    used_vertices = sorted(used_vertices)  # Keep unique, sorted indices
    vertex_map = {old_idx: new_idx for new_idx, old_idx in enumerate(used_vertices)}
    filtered_vertices = vor.vertices[used_vertices]

    # **Re-index faces to match the new filtered vertex list**
    filtered_faces = [[vertex_map[v] for v in face] for face in valid_faces]

    return filtered_vertices, filtered_faces



final_mesh = get_zero_crossing_mesh_3d(sites, model)

ps.register_surface_mesh("Zero-Crossing faces", final_mesh[0], final_mesh[1])
ps.register_point_cloud("Mesh vertices", final_mesh[0])

ps.show()



In [ ]:
def export_visualisation_3d():
    import imageio
    img_buffer_mesh = []
    img_buffer_model = []
    for i in range(int(max_iter/10)+1):
        epoch = i*10
        site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
        model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
        
        print("mesh of epoch: ", epoch)
        
        model.load_state_dict(torch.load(model_file_path))
    
        current_mesh = get_zero_crossing_mesh_3d(torch.load(site_file_path), model)
        ps.remove_all_structures()
        ps.register_surface_mesh("Zero-Crossing faces", current_mesh[0], current_mesh[1])
        ps.register_point_cloud("Mesh vertices", current_mesh[0])
        img_buffer_mesh.append(ps.screenshot_to_buffer(transparent_bg=False))
        
        ps.remove_all_structures()
        polyscope_sdf(model)
        img_buffer_model.append(ps.screenshot_to_buffer(transparent_bg=False))


    imageio.mimsave(f'{destination}{max_iter}_3d_optimization_mesh.gif',img_buffer_mesh, fps=1, duration=1, loop=0)
    imageio.mimsave(f'{destination}{max_iter}_3d_optimization_sdf.gif', img_buffer_model, fps=1, duration=1, loop=0)

export_visualisation_3d()

mesh of epoch:  0
mesh of epoch:  10
mesh of epoch:  20
mesh of epoch:  30
mesh of epoch:  40
mesh of epoch:  50
mesh of epoch:  60
mesh of epoch:  70
mesh of epoch:  80
mesh of epoch:  90
mesh of epoch:  100
